In [ ]:
import glob
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, jaccard_score, precision_score, recall_score, accuracy_score
import os
from datetime import datetime
from tqdm import tqdm
from PIL import Image
from skimage.io import imread, imsave
from ModelArchitecture.DiceLoss import dice_metric_loss

In [ ]:
# Define function to save and load numpy arrays
def save_numpy_arrays(images, masks, dataset_name):
    np.save(f"resized_data/{dataset_name}_images.npy", images)
    np.save(f"resized_data/{dataset_name}_masks.npy", masks)


def load_numpy_arrays(dataset_name):
    images = np.load(f"resized_data/{dataset_name}_images.npy")
    masks = np.load(f"resized_data/{dataset_name}_masks.npy")
    return images, masks


# Define function to load data and resize images and masks
def load_data(image_dir, mask_dir, img_height, img_width, images_to_be_loaded, dataset_name):

    # Check if the resized data already exists
    if os.path.exists(f"resized_data/{dataset_name}_images.npy") and os.path.exists(f"resized_data/{dataset_name}_masks.npy"):
        print(f"Loading preprocessed data for {dataset_name}")
        return load_numpy_arrays(dataset_name)

    if dataset_name == "Kvasir-SEG":
        test_ids = glob.glob(f"{image_dir}/*.jpg")
    elif dataset_name == "CVC-ClinicDB":
        test_ids = glob.glob(f"{image_dir}/*.tif")
    elif dataset_name == "CVC-ColonDB" or dataset_name == "ETIS-LaribpolypDB":
        test_ids = glob.glob(f"{image_dir}/*.png")

    print(test_ids)

    if images_to_be_loaded == -1:
        images_to_be_loaded = len(test_ids)

    images = np.zeros((images_to_be_loaded, img_height, img_width, 3), dtype=np.float32)
    masks = np.zeros((images_to_be_loaded, img_height, img_width), dtype=np.uint8)

    print(f"Resizing training images and masks: {images_to_be_loaded}")
    for n, id_ in tqdm(enumerate(test_ids)):
        if n == images_to_be_loaded:
            break

        image_path = id_

        if "images" in image_path:
            mask_path = image_path.replace("images", "masks")
        elif "Original" in image_path:
            mask_path = image_path.replace("Original", "Ground Truth")

        resized_image_path = image_path.replace("original_data", "resized_data")
        resized_mask_path = mask_path.replace("original_data", "resized_data")

        # Check if the image has been resized
        if os.path.exists(resized_image_path) and os.path.exists(resized_mask_path):
            image = imread(resized_image_path)
            mask_ = imread(resized_mask_path)
        else:
            image = imread(image_path)
            mask_ = imread(mask_path)

            pillow_image = Image.fromarray(image)
            pillow_image = pillow_image.resize((img_height, img_width))
            image = np.array(pillow_image)
            imsave(resized_image_path, image)

            pillow_mask = Image.fromarray(mask_)
            pillow_mask = pillow_mask.resize((img_height, img_width), Image.Resampling.LANCZOS)
            mask_ = np.array(pillow_mask)
            imsave(resized_mask_path, mask_)

        images[n] = image / 255.0

        mask = np.zeros((img_height, img_width), dtype=np.bool_)
        for i in range(img_height):
            for j in range(img_width):
                if np.any(mask_[i, j] >= 127):
                    mask[i, j] = 1

        masks[n] = mask

    masks = np.expand_dims(masks, axis=-1)

    # Save the preprocessed data
    save_numpy_arrays(images, masks, dataset_name)

    return images, masks

In [ ]:
img_size = 352


# Define function to evaluate model
def evaluate_model(model_path, dataset_name, image_dir, mask_dir):
    # Load data
    images, masks = load_data(image_dir, mask_dir, img_size, img_size, -1, dataset_name)

    # Load model
    model = tf.keras.models.load_model(model_path, custom_objects={"dice_metric_loss": dice_metric_loss})

    # Make predictions
    predictions = model.predict(images, batch_size=4)

    # Calculate metrics
    f1 = f1_score(np.ndarray.flatten(np.array(masks, dtype=bool)), np.ndarray.flatten(predictions > 0.5))
    jaccard = jaccard_score(np.ndarray.flatten(np.array(masks, dtype=bool)), np.ndarray.flatten(predictions > 0.5))
    precision = precision_score(np.ndarray.flatten(np.array(masks, dtype=bool)), np.ndarray.flatten(predictions > 0.5))
    recall = recall_score(np.ndarray.flatten(np.array(masks, dtype=bool)), np.ndarray.flatten(predictions > 0.5))
    accuracy = accuracy_score(np.ndarray.flatten(np.array(masks, dtype=bool)), np.ndarray.flatten(predictions > 0.5))

    # Print results
    print(f"Results for {dataset_name} using model {model_path}:")
    print(f"F1 Score: {f1}")
    print(f"Jaccard Score: {jaccard}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"Accuracy: {accuracy}")

    # Save results to file
    result_file = f"results_{os.path.basename(model_path)}_{dataset_name}.txt"
    with open(result_file, "w") as f:
        f.write(f"F1 Score: {f1}\n")
        f.write(f"Jaccard Score: {jaccard}\n")
        f.write(f"Precision: {precision}\n")
        f.write(f"Recall: {recall}\n")
        f.write(f"Accuracy: {accuracy}\n")

    return f1, jaccard, precision, recall, accuracy

In [ ]:
# Define function to plot metrics and save the plots
def plot_metrics(metrics, dataset_names, model_name, save_dir):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    plt.figure(figsize=(15, 5))

    for metric_name, metric_values in metrics.items():
        plt.plot(dataset_names, metric_values, label=metric_name)

    plt.xlabel("Dataset")
    plt.ylabel("Score")
    plt.title(f"Model {model_name} Metrics Across Different Datasets")
    plt.legend()

    # Generate a filename with datetime to avoid overwriting
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    plot_filename = f"{model_name}_metrics_{timestamp}.png"
    plot_filepath = os.path.join(save_dir, plot_filename)

    plt.savefig(plot_filepath)
    plt.show()

    print(f"Plot saved to {plot_filepath}")

In [ ]:
# Define datasets and models
datasets = {
    "Kvasir-SEG": {
        "image_dir": "original_data/Kvasir-SEG/images",
        "mask_dir": "original_data/Kvasir-SEG/masks",
    },
    "CVC-ClinicDB": {
        "image_dir": "original_data/CVC-ClinicDB/Original",
        "mask_dir": "original_data/CVC-ClinicDB/Ground Truth",
    },
    "CVC-ColonDB": {
        "image_dir": "original_data/CVC-ColonDB/images",
        "mask_dir": "original_data/CVC-ColonDB/masks",
    },
    "ETIS-LaribpolypDB": {
        "image_dir": "original_data/ETIS-LaribpolypDB/images",
        "mask_dir": "original_data/ETIS-LaribpolypDB/masks",
    },
}

# Define datasets and models
resized_datasets = {
    "Kvasir-SEG": {
        "image_dir": "resized_data/Kvasir-SEG/images",
        "mask_dir": "resized_data/Kvasir-SEG/masks",
    },
    "CVC-ClinicDB": {
        "image_dir": "resized_data/CVC-ClinicDB/Original",
        "mask_dir": "resized_data/CVC-ClinicDB/Ground Truth",
    },
    "CVC-ColonDB": {
        "image_dir": "resized_data/CVC-ColonDB/images",
        "mask_dir": "resized_data/CVC-ColonDB/masks",
    },
    "ETIS-LaribpolypDB": {
        "image_dir": "resized_data/ETIS-LaribpolypDB/images",
        "mask_dir": "resized_data/ETIS-LaribpolypDB/masks",
    },
}

for dataset_name, dataset_paths in resized_datasets.items():
    if not os.path.exists(dataset_paths["image_dir"]):
        os.makedirs(dataset_paths["image_dir"])
    if not os.path.exists(dataset_paths["mask_dir"]):
        os.makedirs(dataset_paths["mask_dir"])

models = [
    "saved_model/kvasir-17",
    "saved_model/kvasir-34",
    "saved_model/cvc-clinicdb-17",
    "saved_model/cvc-clinicdb-34",
    "saved_model/cvc-colondb-17",
    "saved_model/cvc-colondb-34",
    "saved_model/etis-laribpolypdb-17",
    "saved_model/etis-laribpolypdb-34",
]

# Directory to save the plots
plot_save_dir = "plots"

# Evaluate models on different datasets
for model_path in models:
    metrics = {
        "F1 Score": [],
        "Jaccard Score": [],
        "Precision": [],
        "Recall": [],
        "Accuracy": [],
    }
    dataset_names = []

    for dataset_name, dataset_paths in datasets.items():
        print(f"Using model {os.path.basename(model_path)} to evaluate {dataset_name} dataset...")

        image_dir = dataset_paths["image_dir"]
        mask_dir = dataset_paths["mask_dir"]

        f1, jaccard, precision, recall, accuracy = evaluate_model(model_path, dataset_name, image_dir, mask_dir)

        metrics["F1 Score"].append(f1)
        metrics["Jaccard Score"].append(jaccard)
        metrics["Precision"].append(precision)
        metrics["Recall"].append(recall)
        metrics["Accuracy"].append(accuracy)

        dataset_names.append(dataset_name)

    # Plot metrics
    plot_metrics(metrics, dataset_names, os.path.basename(model_path), plot_save_dir)